In [ ]:
import re
import os
import re 
import pandas as pd
import numpy as np
from pymatgen.core import Structure
from automatminer import AutoFeaturizer
import warnings

warnings.filterwarnings('ignore')

In [ ]:
def load_data(csv_file): 
    return pd.read_csv(csv_file,
                       encoding = 'utf-8') 
data_form = load_data(r"K_origin.csv") 
data_form.head()

In [ ]:
cif_dir = r'./K_cifs'
cif_files= os.listdir(cif_dir)

c=[]
for f in cif_files:
    if f.endswith(".cif"):
        
        f = re.sub("\D", "", f)
        c.append(f)
c_1 = sorted(list(map(int, c)))
new_cif_files = [(str(i)+'.cif') for i in c_1]
cif_paths = [os.path.join(cif_dir, cif) for cif in new_cif_files]


structures = [Structure.from_file(cif) for cif in cif_paths]
structure_df = pd.DataFrame({"structure": structures})

In [ ]:
data_form['structure'] = structure_df
data = data_form
data.head()

In [ ]:
df = data.loc[:, ['structure', 'target']]
df.head()

In [ ]:
featurizer = AutoFeaturizer(preset="express")

featurizer.fit(df, target="target")

featurizer_df = featurizer.transform(df, target="target")

In [ ]:
featurizer_df.head()

In [ ]:
df = featurizer_df.drop(['compound possible', 'is_centrosymmetric'], axis=1)
df

In [28]:
df['crystal_system'] = df['crystal_system'].map({
    'hexagonal' : 2, 
    'cubic' : 1, 
    'trigonal' : 7, 
    'tetragonal' : 4, 
    'orthorhombic' : 5, 
    'triclinic' : 3, 
    'monoclinic' : 6
}).astype(int)

In [ ]:
df[['nelements', 'nsites', 'volume', 'density','density_atomic', 'uncorrected_energy_per_atom',
       'energy_per_atom', 'formation_energy_per_atom', 'energy_above_hull',
       'band_gap', 'efermi', 'spacegroupnumber']] = data_form[['nelements', 'nsites', 'volume', 'density','density_atomic', 'uncorrected_energy_per_atom',
       'energy_per_atom', 'formation_energy_per_atom', 'energy_above_hull',
       'band_gap', 'efermi', 'spacegroupnumber']]

In [ ]:
data_set = df
X = data_set.drop('GVRH',axis=1)
y = data_set['GVRH']

### Training the model

In [ ]:
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error


lgbm_pipeline = LGBMRegressor(n_estimators=1800,num_leaves=31)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 68)
lgbm_pipeline.fit(X_train, y_train)
print('test R2 = ' + str(lgbm_pipeline.score(X_test, y_test)))
print('test MAE = ' + str(mean_absolute_error(y_true = y_test, y_pred = lgbm_pipeline.predict(X_test))))
print('test RMSE = ' + str(np.sqrt(mean_squared_error(y_true = y_test, y_pred = lgbm_pipeline.predict(X_test)))))

### Feature selection

##### Get the top 100 features ranked by feature importance

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

feature_importances = lgbm_pipeline.feature_importances_

feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importances})

feature_importance_df['Importance'] /= feature_importance_df['Importance'].sum()

feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

top_n = 100
top_features = feature_importance_df.head(top_n)

others_importance = feature_importance_df.tail(len(feature_importance_df) - top_n)['Importance'].sum()

others_row = pd.DataFrame({'Feature': ['others'], 'Importance': [others_importance]})
top_features = pd.concat([top_features, others_row], ignore_index=True)

plt.figure(figsize=(12, 8))
bars = plt.barh(top_features['Feature'], top_features['Importance'], color='skyblue')
plt.xlabel('Normalized Importance', fontsize=14)
plt.ylabel('Feature', fontsize=14)

plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.gca().invert_yaxis()

for bar in bars:
    plt.text(
        bar.get_width() + 0.001, 
        bar.get_y() + bar.get_height() / 2, 
        f'{bar.get_width():.4f}', 
        va='center', 
        fontsize=12
    )

plt.show()


### Select key features through hierarchical clustering

In [ ]:
from scipy.cluster.hierarchy import linkage, fcluster
from sklearn.model_selection import cross_val_score, KFold

# Calculate correlation matrix
corr = X.corr().abs()
corr.fillna(0, inplace=True)

# Convert to distance matrix
distance_matrix = 1 - corr
distance_matrix = np.nan_to_num(distance_matrix, nan=0.0, posinf=1.0, neginf=0.0)

# Hierarchical clustering using Ward's method
linkage_matrix = linkage(distance_matrix, method='ward')

best_threshold = None
best_score = -np.inf
best_mae = float('inf')
best_rmse = float('inf')

kf = KFold(n_splits=10, shuffle=True, random_state=68)

# Loop through different thresholds
for threshold in np.arange(0, 2.0, 0.05):
    clusters = fcluster(linkage_matrix, threshold, criterion='distance')
    selected_features = []
    
    for cluster_id in np.unique(clusters):
        cluster_features = corr.columns[clusters == cluster_id]
        cluster_importances = feature_importance_df[feature_importance_df['Feature'].isin(cluster_features)]
        
        if not cluster_importances.empty:
            representative_feature = cluster_importances.loc[cluster_importances['Importance'].idxmax(), 'Feature']
            selected_features.append(representative_feature)

    # Create new feature matrix
    X_selected = X[selected_features]

    # Train and evaluate using 10-fold cross-validation
    lgbm = LGBMRegressor(n_estimators=1800, num_leaves=31)

    r2_scores = cross_val_score(lgbm, X_selected, y, cv=kf, scoring='r2')
    mae_scores = -cross_val_score(lgbm, X_selected, y, cv=kf, scoring='neg_mean_absolute_error')
    rmse_scores = np.sqrt(-cross_val_score(lgbm, X_selected, y, cv=kf, scoring='neg_mean_squared_error'))

    r2_mean = r2_scores.mean()
    mae_mean = mae_scores.mean()
    rmse_mean = rmse_scores.mean()

    # Check if this is the best model
    if r2_mean > best_score:
        best_score = r2_mean
        best_mae = mae_mean
        best_rmse = rmse_mean
        best_threshold = threshold

    print(f'Threshold: {threshold:.2f}, R2: {r2_mean:.4f}, MAE: {mae_mean:.4f}, RMSE: {rmse_mean:.4f}')

# Output best results
print(f'Best Threshold: {best_threshold:.1f}')
print(f'Best R2: {best_score:.4f}')
print(f'Best MAE: {best_mae:.4f}')
print(f'Best RMSE: {best_rmse:.4f}')

##### Carry out clustering to select important features

In [ ]:
# Extract clusters
threshold = k  # k can be obtained from the paper
clusters = fcluster(linkage_matrix, threshold, criterion='distance')
selected_features_res = []
for cluster_id in np.unique(clusters):
    # Get features in current cluster
    cluster_features = corr.columns[clusters == cluster_id]
    
    # Find importance of these features from feature_importance_df
    cluster_importances = feature_importance_df[feature_importance_df['Feature'].isin(cluster_features)]
    
    # Select feature with highest importance contribution
    if not cluster_importances.empty:
        # Find feature with highest importance
        representative_feature = cluster_importances.loc[cluster_importances['Importance'].idxmax(), 'Feature']
        selected_features_res.append(representative_feature)

# Update dataset with selected features
X = df[selected_features_res]
X.head()

In [ ]:
X.columns